In [1]:
import json
import os
import pandas as pd
import re

In [2]:
os.chdir('/mnt/data/hong/courses/varianceAnalysis/')
# with open("data/ADA_uniprot.json", "r") as file:
#     uniprot = file.read()
#     ins = re.findall(r'(p\.[A-Z]{1}[a-z]+\d+_[A-Z]{1}[a-z]+\d+ins[A-Z]{1}[a-z]+)"',uniprot)
#     deletions = re.findall(r'(p\.[A-Z]{1}[a-z]+\d+del)',uniprot)

The following block is to generate random p. insertions and deletions  
and is not necessary for the assignment

In [7]:
one2all ={'A': ('A', 'ALA', 'alanine'),
    'R': ('R', 'ARG', 'arginine'),
    'N': ('N', 'ASN', 'asparagine'),
    'D': ('D', 'ASP', 'aspartic acid'),
    'C': ('C', 'CYS', 'cysteine'),
    'Q': ('Q', 'GLN', 'glutamine'),
    'E': ('E', 'GLU', 'glutamic acid'),
    'G': ('G', 'GLY', 'glycine'),
    'H': ('H', 'HIS', 'histidine'),
    'I': ('I', 'ILE', 'isoleucine'),
    'L': ('L', 'LEU', 'leucine'),
    'K': ('K', 'LYS', 'lysine'),
    'M': ('M', 'MET', 'methionine'),
    'F': ('F', 'PHE', 'phenylalanine'),
    'P': ('P', 'PRO', 'proline'),
    'S': ('S', 'SER', 'serine'),
    'T': ('T', 'THR', 'threonine'),
    'W': ('W', 'TRP', 'tryptophan'),
    'Y': ('Y', 'TYR', 'tyrosine'),
    'V': ('V', 'VAL', 'valine'),
    'X': ('X', 'GLX', 'glutaminx'),
    'Z': ('Z', 'GLI', 'glycine'),
    'J': ('J', 'NLE', 'norleucine'),
    'U': ('U', 'CYC', 'cysteinc')
}

In [6]:
## read in fa
with open("data/ADA.fa", "r") as file:
    fa = file.read()
    fa = fa.split('\n')
    fa = [i for i in fa if i != '']
    fa = fa[1:]
    fa = ''.join(fa)
    fa = list(fa)

In [15]:
## As I didn't find a good reference for insertion or deletoin frequency, 
## and the question of this assignment has nothing to do with the frequency
## but just the functional predictoin "when" it happens
## So these indels are created by random
## I will create 9 random insertions and 9 random deletions
import random
random.seed(404)
def mk_ins(fa, random_ins, ins):
    for i in random_ins:
        aa1 = one2all[fa[i]][1]
        aa2 = one2all[fa[i+1]][1]
        aai = random.choice(list(one2all.keys()))
        insi = f'p.{aa1}{i}_{aa2}{i+1}ins{aai}'
        ins.append(insi)
    return ins

## generate 18 random integers in the range of length of fa

random_ints = random.sample(range(0, len(fa)), 18)
random_ins = random_ints[:9]
random_del = random_ints[9:]
ins_generate = mk_ins(fa, random_ins, ins)

In [17]:
def mk_del(fa, random_del, deletions):
    for i in random_del:
        aa1 = one2all[fa[i]][1]
        deli = f'p.{aa1}{i}del'
        deletions.append(deli)
    return deletions

In [19]:
deletions_generate = mk_del(fa, random_del, deletions)

In [21]:
## write out ins_generate and deletions_generate
## to tsv files
df_ins = pd.DataFrame(ins_generate, columns=['ins'])
df_del = pd.DataFrame(deletions_generate, columns=['del'])
df_ins.to_csv('data/ins.tsv', sep='\t', index=False)
df_del.to_csv('data/del.tsv', sep='\t', index=False)

## from here comes what's used in answering the exercises

In [3]:

import pyhgvs as hgvs
from pyhgvs.utils import read_transcripts
from pyfaidx import Fasta

# Read genome sequence using pyfaidx.
genome = Fasta('/mnt/data/hong/tools/annovar/humandb/hg19_seq/hg19.fa')

# Read RefSeq transcripts into a python dict.
with open('/mnt/data/hong/tools/annovar/humandb/hg19_refGene.txt') as infile:
    transcripts = read_transcripts(infile)

# Provide a callback for fetching a transcript by its name.
def get_transcript(name):
    return transcripts.get(name)

In [8]:
## get codon frequency
def index_containing_substring(the_list, substring):
    for i, s in enumerate(the_list):
        if substring in s:
              return i
    return -1
with open('/mnt/data/hong/tools/annovar/humandb/hg19_refGeneMrna.fa') as fi:
    line_list = fi.readlines()
    line_idx = index_containing_substring(line_list, 'NM_000022')
    print(line_list[line_idx])
    line_start ='a'
    coding_seq = ''
    while line_start != '>':
        coding_seq = coding_seq+ line_list[line_idx+1].strip()
        line_idx += 1
        line_start = line_list[line_idx+1][0]

## need to get the correct transcript sequence

>NM_000022   Comment: this sequence (leftmost exon at chr20:43248162) is generated by ANNOVAR on Thu Feb 22 09:03:55 2024, based on regions specified in humandb/hg19_refGene.txt and sequence files stored at humandb/hg19_seq.



In [6]:
len_transcript = len(coding_seq)

In [7]:
codon = [coding_seq[i:i+3] for i in range(0,len(coding_seq), 3)]

In [14]:
## consider that I might fall short of insertions or deletions
## as some of them might map to other genes close to the gene of interest
## or in the UTR region

In [9]:
# Parse the HGVS name into genomic coordinates and alleles.
## create HGVS names
## 'NG_007385.1':c.1234_1235insACGT
## https://databases.lovd.nl/shared/refseq/ADA_NM_000022.2_codingDNA.html
ng = 'NM_000022'
import random
random.seed(404)
def mk_cins():
    i = 1
    while True:
        start = random.sample(range(1, len_transcript//3-3), 1)[0]*3-1
        end = start +1
        ins_seq = random.sample(codon, 1)[0]
        ins_hgvs = f'{ng}:c.{start}_{end}ins{ins_seq}'
        yield ins_hgvs, i
        i += 1
chroms, offsets, refs, alts = [], [], [], []
with open('data/indel.vcf', 'a') as out_vcf:
    out_vcf.write('#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO\n')
    for ins_names, num in mk_cins():
        if num > 15:
            break
        chrom, offset, ref, alt = hgvs.parse_hgvs_name(ins_names, genome, get_transcript=get_transcript, lazy=True)
        out_vcf.write(f'{chrom}\t{offset}\t{ins_names}\t{ref}\t{alt}\t.\tPASS\t.\n')

In [10]:
#NM_000492.3:c.1438_1440delGGT
def mk_cdel():
    i = 1
    while True:
        th = random.sample(range(1, len_transcript//3-3), 1)[0]
        start = th*3-1
        end = start +2
        del_seq = codon[th]
        del_hgvs = f'{ng}:c.{start}_{end}del{del_seq}'
        yield del_hgvs, i
        i += 1
chroms, offsets, refs, alts = [], [], [], []
with open('data/indel.vcf', 'a') as out_vcf:
    for del_names, num in mk_cdel():
        if num > 15:
            break
        chrom, offset, ref, alt = hgvs.parse_hgvs_name(del_names, genome, get_transcript=get_transcript, lazy=True)
        out_vcf.write(f'{chrom}\t{offset}\t{del_names}\t{ref}\t{alt}\t.\tPASS\t.\n')